# TritonHacks 2025: Introduction to AI/ML Starter Kit Part II: Machine Learning (ML)

Welcome to the Intro to AI/ML Starter Kit for TritonHacks 2025! This is the second of two notebooks in this repo, and it focuses on creating ML models.

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

In [21]:
# Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/AI-ML-StarterKit/healthcare-dataset-stroke-data.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [16]:
# Define X and y

X = df.drop("stroke", axis = 1)
y = df["stroke"]

In [17]:
# Train-Test Spliting

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
KNN = KNeighborsClassifier(n_neighbors = 9)
KNN.fit(X_train, y_train)
y_test = KNN.predict(X_test)

ValueError: could not convert string to float: 'Male'